In [ ]:
# Filesystem & data handling
import os
import glob
import pathlib

import numpy as np
import pandas as pd

# Image processing
import cv2

# TensorFlow (for TFRecord creation & later data pipeline)
import tensorflow as tf

# Visualization (for debugging bounding boxes & annotations)
import matplotlib.pyplot as plt


In [ ]:
def analyze_dataset_distribution(annotations_dir):
    """
    Analyzes the distribution of bounding boxes across a dataset.
    It returns the maximum number of boxes found and visualizes the
    distribution with a histogram.

    Args:
        annotations_dir (str): Path to the directory containing annotation files (.txt).
    
    Returns:
        int: The maximum number of bounding boxes found in a single file.
    """
    if not os.path.isdir(annotations_dir):
        print(f"Error: Directory not found at {annotations_dir}")
        return 0

    box_counts = []
    max_boxes = 0

    print("Analyzing dataset...")
    # Iterate through all files in the directory
    for filename in os.listdir(annotations_dir):
        if filename.endswith(".txt"):
            filepath = os.path.join(annotations_dir, filename)
            try:
                # Count the number of lines in the file
                with open(filepath, 'r') as f:
                    num_lines = sum(1 for line in f)
                
                box_counts.append(num_lines)
                
                # Update the maximum count if necessary
                if num_lines > max_boxes:
                    max_boxes = num_lines
            except Exception as e:
                print(f"Warning: Could not read file {filepath}. Skipping. Error: {e}")
    
    if not box_counts:
        print("No annotation files found or all files were empty.")
        return 0
    
    print(f"Analysis complete. Found {len(box_counts)} annotation files.")

    # Visualize the distribution with a histogram
    plt.figure(figsize=(10, 6))
    plt.hist(box_counts, bins=range(0, max_boxes + 2), edgecolor='black', alpha=0.7)
    plt.axvline(max_boxes, color='red', linestyle='dashed', linewidth=2, label=f'Max Boxes: {max_boxes}')
    plt.title('Distribution of Bounding Boxes per Image')
    plt.xlabel('Number of Bounding Boxes')
    plt.ylabel('Number of Images')
    plt.xticks(np.arange(0, max_boxes + 2, step=1))
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    
    # Save the plot
    plot_filename = 'box_distribution.png'
    plt.savefig(plot_filename)
    print(f"Distribution plot saved as '{plot_filename}'")
    
    return max_boxes


In [ ]:
max_boxes_found = analyze_dataset_distribution('/kaggle/input/rdd-2022/RDD_SPLIT/train/labels')

In [ ]:
def parse_image_and_labels(image_path, annot_path):
    """
    Reads image and annotation, prepares data.
    This version uses only TensorFlow for file I/O to avoid
    the dependency on the cv2 library.

    Args:
        image_path (str): Path to the image file.
        annot_path (str): Path to the annotation file.
        
    Returns:
        dict with image metadata and annotations, or None if an error occurs.
    """
    try:
        # Read the image as compressed bytes
        image_bytes = tf.io.read_file(image_path)
        
        # Decode the image to get its shape.
        # The image is immediately cast back to bytes to save memory.
        image_decoded = tf.io.decode_image(image_bytes, channels=3)
        height, width, _ = image_decoded.shape
        
        if height is None or width is None:
            raise ValueError("Could not decode image to get shape.")

    except tf.errors.OpError as e:
        print(f"Warning: Could not read or decode image {image_path}. Skipping. Error: {e}")
        return None
    except ValueError as e:
        print(f"Warning: {e} for image {image_path}. Skipping.")
        return None

    # --- Read annotations ---
    bboxes = []
    labels = []
    
    if os.path.exists(annot_path) and os.path.getsize(annot_path) > 0:
        with open(annot_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                    
                try:
                    class_id = int(parts[0])
                    x_center, y_center, box_w, box_h = map(float, parts[1:])
                except ValueError:
                    print(f"Warning: Skipping malformed line in {annot_path}: '{line.strip()}'")
                    continue
                
                # Convert YOLO format to normalized (xmin, ymin, xmax, ymax)
                xmin = x_center - (box_w / 2)
                ymin = y_center - (box_h / 2)
                xmax = x_center + (box_w / 2)
                ymax = y_center + (box_h / 2)
                
                # Clip coordinates to the [0, 1] range
                xmin, ymin = max(0.0, xmin), max(0.0, ymin)
                xmax, ymax = min(1.0, xmax), min(1.0, ymax)
                
                bboxes.append([xmin, ymin, xmax, ymax])
                labels.append(class_id)
                
    return {
        "filename": os.path.basename(image_path),
        "height": height,
        "width": width,
        "image_bytes": image_bytes,
        "bboxes": bboxes,
        "labels": labels
    }


In [ ]:
import matplotlib.patches as patches
def visualize_parsed_record(record, class_names=None):
    """
    Visualize one parsed record (image + bounding boxes).
    
    Args:
        record (dict): Output from parse_image_and_labels()
        class_names (list or dict): Optional mapping {id: name} for labels
    """
    # Decode image bytes back to array
    # Read the image as compressed bytes
    image = tf.image.decode_jpeg(record['image_bytes'], channels=3)  # Specifically for JPEG
    image_array = image.numpy()

    height, width = record["height"], record["width"]
    print(record["bboxes"])
    fig, ax = plt.subplots(1, figsize=(8, 6))
    ax.imshow(image_array)
    ax.set_title(record["filename"])
    
    # Draw bounding boxes
    for i, (xmin, ymin, xmax, ymax) in enumerate(record["bboxes"]):
        # Convert from normalized to absolute pixel coords
        xmin_px, ymin_px = int(xmin * width), int(ymin * height)
        xmax_px, ymax_px = int(xmax * width), int(ymax * height)

        rect = patches.Rectangle(
            (xmin_px, ymin_px),
            xmax_px - xmin_px,
            ymax_px - ymin_px,
            linewidth=2,
            edgecolor="red",
            facecolor="none"
        )
        ax.add_patch(rect)

        # Add label
        if len(record["labels"]) > i:
            class_id = record["labels"][i]
            label = class_names[class_id] if class_names else str(class_id)
            ax.text(
                xmin_px, ymin_px - 5, label,
                color="yellow", fontsize=10,
                bbox=dict(facecolor="red", alpha=0.5, edgecolor="none")
            )

    plt.axis("off")
    plt.show()


In [ ]:
# Example: Pick one image + annotation
img_path = "/kaggle/input/rdd-2022/RDD_SPLIT/train/images/China_Drone_000001.jpg"
annot_path = "/kaggle/input/rdd-2022/RDD_SPLIT/train/labels/China_Drone_000001.txt"
record = parse_image_and_labels(img_path, annot_path)


In [ ]:
visualize_parsed_record(record)

In [ ]:
def _bytes_feature(value):
    """Return a bytes_list from a string/byte."""
    if isinstance(value, type(tf.constant(0))):  # EagerTensor check
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Return a float_list from a list of floats."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    """Return an int64_list from a list of ints."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


In [ ]:
def create_tf_example(record):
    """
    Convert parsed image/annotation dict into a tf.train.Example.

    Args:
        record (dict): Output from parse_image_and_labels()

    Returns:
        tf.train.Example
    """
    filename    = record["filename"]
    height      = record["height"]
    width       = record["width"]
    image_bytes = record["image_bytes"]
    bboxes      = record["bboxes"]   # list of [xmin, ymin, xmax, ymax] normalized
    labels      = record["labels"]   # list of int class IDs

    # Split bboxes into separate lists (stay empty if no objects)
    xmins, ymins, xmaxs, ymaxs = [], [], [], []
    for (xmin, ymin, xmax, ymax) in bboxes:
        xmins.append(xmin)
        ymins.append(ymin)
        xmaxs.append(xmax)
        ymaxs.append(ymax)

    # Build feature dict
    feature_dict = {
        "image/height": _int64_feature([height]),
        "image/width": _int64_feature([width]),
        "image/filename": _bytes_feature(filename.encode("utf8")),
        "image/encoded": _bytes_feature(image_bytes),
        "image/object/bbox/xmin": _float_feature(xmins),
        "image/object/bbox/ymin": _float_feature(ymins),
        "image/object/bbox/xmax": _float_feature(xmaxs),
        "image/object/bbox/ymax": _float_feature(ymaxs),
        "image/object/class/label": _int64_feature(labels),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature_dict))


In [ ]:
import math
def write_tfrecord(image_dir: str, annot_dir: str, output_dir: str, num_shards: int):
    """
    Creates TFRecord shards from image and annotation directories.

    Args:
        image_dir (str): Path to the images folder.
        annot_dir (str): Path to the annotations folder.
        output_dir (str): Path where the .tfrecord files will be saved.
        num_shards (int): The total number of TFRecord shards to create.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get all image paths and sort them to ensure consistent sharding
    all_img_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")))
    total_images = len(all_img_paths)
    print(total_images)
    
    # Calculate the number of images per shard
    shard_size = math.ceil(total_images / num_shards)
    print(shard_size)
    
    print(f"Starting TFRecord sharding into {num_shards} files...")

    # Iterate through each shard
    for i in range(num_shards):
        # Calculate the start and end indices for this shard
        start_index = i * shard_size
        end_index = min((i + 1) * shard_size, total_images)
        
        # Get the subset of image paths for the current shard
        shard_img_paths = all_img_paths[start_index:end_index]
        
        # Format the output filename to indicate the shard number
        # e.g., 'data-00000-of-00005.tfrecord'
        shard_filename = f"data-{i:05d}-of-{num_shards:05d}.tfrecord"
        output_path = os.path.join(output_dir, shard_filename)
        
        num_written = 0
        with tf.io.TFRecordWriter(output_path) as writer:
            for img_path in shard_img_paths:
                try:
                    fname = os.path.splitext(os.path.basename(img_path))[0]
                    annot_path = os.path.join(annot_dir, fname + ".txt")

                    # Use your existing helper functions
                    record = parse_image_and_labels(img_path, annot_path)
                    tf_example = create_tf_example(record)

                    # Write to TFRecord
                    writer.write(tf_example.SerializeToString())
                    num_written += 1
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
                    
        print(f"✅ Shard {i + 1}/{num_shards}: {num_written} samples written to {output_path}")
        
    print("✅ All TFRecord shards have been created.")


In [ ]:
# # Train set
write_tfrecord(
    image_dir="/kaggle/input/rdd-2022/RDD_SPLIT/train/images",
    annot_dir="/kaggle/input/rdd-2022/RDD_SPLIT/train/labels",
    output_dir="train_tfrecords",
    num_shards = 100
)

# Validation set
write_tfrecord(
    image_dir="/kaggle/input/rdd-2022/RDD_SPLIT/val/images",
    annot_dir="/kaggle/input/rdd-2022/RDD_SPLIT/val/labels",
    output_dir="val_tfrecrods",
    num_shards = 10
)

# Test set
write_tfrecord(
    image_dir="/kaggle/input/rdd-2022/RDD_SPLIT/test/images",
    annot_dir="/kaggle/input/rdd-2022/RDD_SPLIT/test/labels",
    output_dir="test_tfrecords",
    num_shards = 10
)


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub


In [ ]:
feature_description = {
    "image/filename": tf.io.FixedLenFeature([], tf.string), # Add this line
    "image/height": tf.io.FixedLenFeature([], tf.int64),
    "image/width": tf.io.FixedLenFeature([], tf.int64),
    "image/encoded": tf.io.FixedLenFeature([], tf.string),
    "image/object/bbox/xmin": tf.io.VarLenFeature(tf.float32),
    "image/object/bbox/ymin": tf.io.VarLenFeature(tf.float32),
    "image/object/bbox/xmax": tf.io.VarLenFeature(tf.float32),
    "image/object/bbox/ymax": tf.io.VarLenFeature(tf.float32),
    "image/object/class/label": tf.io.VarLenFeature(tf.int64)
}

In [ ]:
MAX_BOXES = max_boxes_found
SENTINEL_BOX = [0.0, 0.0, 0.0, 0.0]  # empty box
BACKGROUND_LABEL = -1  # empty box label
IMAGE_SIZE = 512
def parse_tfrecord(example_proto):
    """
    Parses a single tf.train.Example from a TFRecord file, using a globally
    defined `feature_description` dictionary.

    Args:
        example_proto: A serialized tf.train.Example.

    Returns:
        A tuple containing the processed image and a dictionary with
        the normalized and padded bounding boxes, labels, and the filename.
    """
    # Parse the serialized example using the globally available feature_description.
    features = tf.io.parse_single_example(example_proto, feature_description)

    # Decode image from JPEG format and convert to float32, normalizing
    # pixel values to the range [0, 1].
    image = tf.image.decode_jpeg(features["image/encoded"], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])


    
    # Get the filename from the parsed features.
    filename = features["image/filename"]

    # Convert sparse bounding box coordinate tensors to dense tensors.
    xmin = tf.sparse.to_dense(features["image/object/bbox/xmin"])
    ymin = tf.sparse.to_dense(features["image/object/bbox/ymin"])
    xmax = tf.sparse.to_dense(features["image/object/bbox/xmax"])
    ymax = tf.sparse.to_dense(features["image/object/bbox/ymax"])

    # Stack the coordinates to form a single tensor of shape [num_boxes, 4].
    # The order is ymin, xmin, ymax, xmax.
    boxes = tf.stack([ymin, xmin, ymax, xmax], axis=1)

    # Convert sparse label tensor to a dense tensor and cast to int32.
    # int32 is a common and efficient type for label handling in TensorFlow.
    labels = tf.sparse.to_dense(features["image/object/class/label"])
    labels = tf.cast(labels, tf.int32)

    num_boxes = tf.shape(boxes)[0]

    # Pad boxes and labels to a fixed size (MAX_BOXES) using a conditional operation.
    # This ensures that all tensors in the batch have the same shape.
    pad_size = MAX_BOXES - num_boxes
    
    # If the number of boxes is less than MAX_BOXES, pad with sentinel values.
    # If it's more, truncate the tensors to the maximum allowed.
    boxes = tf.cond(pad_size > 0,
                    lambda: tf.concat([boxes, tf.tile([SENTINEL_BOX], [pad_size,1])], axis=0),
                    lambda: boxes[:MAX_BOXES])
    labels = tf.cond(pad_size > 0,
                     lambda: tf.concat([labels, tf.fill([pad_size], BACKGROUND_LABEL)], axis=0),
                     lambda: labels[:MAX_BOXES])

    # Return the image and a dictionary containing the processed data, including the filename.
    return image, {"boxes": boxes, "classes": labels}

In [ ]:
import os
# Define the paths to your directories
train_dir = '/kaggle/working/train_tfrecords'
val_dir = '/kaggle/working/val_tfrecrods'

# 1. Get the list of filenames and create full paths
train_files = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]
val_files = [os.path.join(val_dir, f) for f in os.listdir(val_dir)]


# 2. Create two separate TFRecordDataset objects
train_dataset = tf.data.TFRecordDataset(train_files)
val_dataset = tf.data.TFRecordDataset(val_files)

# 3. Apply the parsing function to each dataset independently
train_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
for img, targets in train_dataset.take(1):
    print("Image shape:", img.shape)
    print("Boxes:", targets["boxes"])
    print("ckasses:", targets["classes"])



In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# This dictionary should be consistent with the labels used in your TFRecord creation.
LABEL_MAP = {
    0: 'longitudinal crack',
    1: 'transverse crack',
    2: 'alligator crack',
    3: 'other corruption',
    4: 'Pothole',
    5: 'Background'  # Background label for padding
}

def visualize_boxes(example):
    """
    Visualizes an image from a parsed TFRecord example with bounding boxes.
    
    This function takes a single example from the dataset, extracts the image and
    bounding box information, and displays the image with the boxes drawn on it.
    It is useful for debugging and verifying that the data has been parsed
    and normalized correctly.
    
    Args:
        example: A tuple containing the image tensor and a dictionary of features.
                 Expected format: (image_tensor, {'boxes': boxes_tensor, 'labels': labels_tensor, 'filename': filename_tensor})
    """
    # Unpack the example
    image, features = example
    boxes = features['boxes']
    labels = features['classes']

    # Convert tensors to NumPy arrays
    # The image is normalized to [0, 1] in the parsing function, so we scale it back to [0, 255]
    image_np = (image.numpy() * 255).astype(np.uint8)
    boxes_np = boxes.numpy()
    labels_np = labels.numpy()
    
    # Get image dimensions to convert normalized box coordinates to pixel coordinates
    height, width, _ = image_np.shape

    # Create a figure and axes
    fig, ax = plt.subplots(1)
    
    # Display the image
    ax.imshow(image_np)




    # Iterate through each bounding box and draw it
    for i, box in enumerate(boxes_np):
        label_id = labels_np[i]

        # Filter out the padded boxes where the label is the background label (5)
        if label_id != 5:
            ymin, xmin, ymax, xmax = box

            # Convert normalized coordinates to pixel coordinates
            box_xmin = xmin * width
            box_ymin = ymin * height
            box_width = (xmax - xmin) * width
            box_height = (ymax - ymin) * height

            # Create a Rectangle patch
            rect = patches.Rectangle(
                (box_xmin, box_ymin), 
                box_width, 
                box_height, 
                linewidth=2, 
                edgecolor='r', 
                facecolor='none'
            )

            # Add the patch to the Axes
            ax.add_patch(rect)
            
            # Add a text label above the box
            label_text = LABEL_MAP.get(label_id)
            plt.text(box_xmin, box_ymin - 5, label_text, 
                     color='white', 
                     bbox=dict(facecolor='red', alpha=0.8),
                     fontsize=8)

    # Display the plot
    plt.show()

# To use these functions, you would first create a TFRecordDataset,
# then map the `parse_tfrecord_example` function over it, and finally
# pass individual examples to the `visualize_boxes` function.
# Example:
# train_dataset = tf.data.TFRecordDataset(your_tfrecord_files)
# parsed_dataset = train_dataset.map(parse_tfrecord_example)
# for example in parsed_dataset.take(5):
#     visualize_boxes(example)


In [ ]:
for example in train_dataset.take(1):
    visualize_boxes(example)

In [ ]:
import tensorflow as tf

def flip_boxes_left_right(boxes):
    ymin, xmin, ymax, xmax = tf.unstack(boxes, axis=1)
    xmin_new = 1.0 - xmax
    xmax_new = 1.0 - xmin
    return tf.stack([ymin, xmin_new, ymax, xmax_new], axis=1)

def rotate_boxes_90(boxes):
    ymin, xmin, ymax, xmax = tf.unstack(boxes, axis=1)
    new_ymin = 1.0 - xmax
    new_xmin = ymin
    new_ymax = 1.0 - xmin
    new_xmax = ymax
    return tf.stack([new_ymin, new_xmin, new_ymax, new_xmax], axis=1)


def augment_image(image, boxes, classes):
    # build mask for valid boxes (not padding)
    valid_mask = tf.expand_dims(tf.not_equal(classes, -1), -1)
    valid_mask = tf.tile(valid_mask, [1, 4])

    # horizontal flip
    if tf.random.uniform([]) > 0.5:
        image = tf.image.flip_left_right(image)
        boxes = tf.where(valid_mask, flip_boxes_left_right(boxes), boxes)

    # brightness & contrast
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)

    # 90° rotation
    if tf.random.uniform([]) > 0.7:
        image = tf.image.rot90(image)
        boxes = tf.where(valid_mask, rotate_boxes_90(boxes), boxes)

    image = tf.clip_by_value(image, 0., 1.)
    return image, boxes, classes




In [ ]:
def augment_fn(image, targets):
    boxes = targets["boxes"]
    labels = targets["classes"]

    image, boxes, labels = augment_image(image, boxes, labels)

    return image, {"boxes": boxes, "classes": labels}


In [ ]:
BATCH_SIZE = 64  # small for CPU debug
# parsed dataset from Step 1
augmented_dataset = train_dataset.map(augment_fn, num_parallel_calls=tf.data.AUTOTUNE)




In [ ]:
# 2. Add the denormalization step
# This applies the denormalization to each sample individually
augmented_dataset = augmented_dataset.map(lambda img, boxes: (
    tf.cast(img * 255.0, tf.float32), 
    {
        'boxes': boxes['boxes'] * tf.cast(tf.stack([IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE]), tf.float32), 
        'classes': boxes['classes']
    }
))

# 3. Continue with batching and prefetching
augmented_dataset = augmented_dataset.batch(BATCH_SIZE, drop_remainder=True)
augmented_dataset = augmented_dataset.prefetch(tf.data.AUTOTUNE)


val_dataset = val_dataset.map(lambda img, boxes: (
    tf.cast(img * 255.0, tf.float32), 
    {
        'boxes': boxes['boxes'] * tf.cast(tf.stack([IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE]), tf.float32), 
        'classes': boxes['classes']
    }
))

# 3. Continue with batching and prefetching
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
for batch_images, batch_targets in augmented_dataset.take(5):
    print("Batch images shape:", batch_images.shape, batch_images.dtype)
    print("Batch boxes shape:", batch_targets["boxes"].shape, batch_targets["boxes"].dtype)
    print("Batch labels shape:", batch_targets["classes"].shape, batch_targets["classes"].dtype)

    # Optional: visualize first image and boxes
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches

    img = batch_images[0].numpy()
    boxes = batch_targets["boxes"][0].numpy()
    
    fig, ax = plt.subplots(1,1)
    ax.imshow(img)
    for box in boxes:
        if (box != [0,0,0,0]).any():  # ignore sentinel boxes
            ymin, xmin, ymax, xmax = box
            rect = patches.Rectangle(
                (xmin*img.shape[1], ymin*img.shape[0]),
                (xmax - xmin)*img.shape[1],
                (ymax - ymin)*img.shape[0],
                linewidth=2, edgecolor='r', facecolor='none'
            )
            ax.add_patch(rect)
    plt.show()


In [ ]:
for images, targets in augmented_dataset.take(1):
    print("image range:", tf.reduce_min(images), tf.reduce_max(images))
    print("boxes example:", targets["boxes"][0][:5])
    print("classes example:", targets["classes"][0][:5])

In [ ]:

# TPU strategy (optional)
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect('local')
    strategy = tf.distribute.TPUStrategy(tpu)
    print("✅ TPU detected")
except:
    strategy = tf.distribute.get_strategy()
    print("✅ Using", strategy)

IMG_SIZE = 512
NUM_CLASSES = 5   # only foreground classes, background handled automatically
BATCH_SIZE = 64
EPOCHS = 100


In [ ]:
from keras_cv.models import RetinaNet, ResNet50Backbone
with strategy.scope():
    backbone = ResNet50Backbone(include_rescaling=True)
    model = RetinaNet(
        backbone=backbone,
        include_rescaling=True,          # we already resized
        bounding_box_format="yxyx",
        num_classes=NUM_CLASSES,
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        classification_loss="focal",
        box_loss="SmoothL1"
    )
    model.summary()


In [ ]:
history = model.fit(
    augmented_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint("retinanet_best.h5",
                                           save_best_only=True,
                                           monitor="val_loss"),
        tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True),
    ]
)


In [ ]:
model2 = tf.keras.models.load_model('retinanet_best.h5')

In [ ]:
# Save the model as a SavedModel
model2.save('retinanet_best_savedmodel', save_format='tf')

print("Model converted successfully to SavedModel format at Model/retinanet_best_savedmodel")